In [112]:
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
#%matplotlib tk
%autosave 180

import sys
sys.path.append('/home/cat/code/gerbil/utils/')
#
import numpy as np
from tqdm import trange

#
#from visualize import Visualize


#
import sleap

#
import cv2
import os

#
from tqdm import tqdm
import parmap

#
import pandas as pd

#
from track.track import make_human_label_only_slp, make_human_label_only_video

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [114]:
################################################################
########### SET FNAMES FOR HUMAN AND ID SWITCH FILES ###########
################################################################
from track.track import DatabaseLoader

#
#root_dir = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/'
root_dir = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/'
fname_database = 'database.xlsx'
human_slp_dir = 'human_labels'
input_dir = 'input_dir'
n_cores = 16
network_type = "Day"   # "Day" or "Night"

# load database class
d = DatabaseLoader(root_dir + fname_database,
                   network_type)
d.nn_type = network_type
d.root_dir = root_dir
d.human_slp_dir = human_slp_dir
d.input_dir = input_dir
d.n_cores = n_cores
d.network_type = network_type
d.track_type = 'feature'
d.recompute_spine_centres = False
d.skip_id_switch_make = False

# make id switch files over all files in /input_dir/ and database.xlsx 
d.make_id_switch_files()

# process the human labeled .slp file to short version
d.fname_slp_human_labels = 'cohort4_day.1000.slp'
d.clean_human_slp()

# merge the id switch and human files

d.subsample=2
d.merge_human_and_id_switches()

#
print ("DONE...")


334it [00:00, 8056.83it/s]


recomputing id switches parallel



100%|██████████████████████████████████████| 114/114 [00:00<00:00, 13412.74it/s]



done running .slp preprocessing and id_switch detection ..., # files:  114

making shortened id_switch .slp files...
ALL SLP VIDS:  [['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_22_17_30_03_197326_compressed_Day_deleted.slp']
 ['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_22_18_36_53_237550_compressed_Day_deleted.slp']
 ['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_23_10_13_20_935110_compressed_Day_deleted.slp']
 ['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_23_11_20_15_845479_compressed_Day_deleted.slp']
 ['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_23_12_27_11_709516_compressed_Day_deleted.slp']
 ['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_23_16_55_36_033685_compressed_Day_deleted.slp']
 ['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_24_08_36_41_

total frames: 1600: 1600it [05:42,  4.67it/s]


# of images saved  1600
Video(filename=/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/Day_merged.avi, shape=(None, None, None, None), backend=MediaVideo)
slp ctrl:  0
slp ctrl:  1000
slp ctrl:  2000
slp ctrl:  3000
slp ctrl:  4000
slp ctrl:  5000
slp ctrl:  6000
slp ctrl:  7000
Loaded video sizes: width, height:  1160.0 830.0
processing:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/Day_human_labels.avi
processing:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/Day_id_switches_cropped.mp4
done...
DONE...


In [14]:
d = np.load('/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_22_16_23_20_653413_compressed_Day.npy')

print (d.shape)

print (np.nanmin(d[:,:,:,0]))
print (np.nanmax(d[:,:,:,0]))
       
print (np.nanmin(d[:,:,:,1]))
print (np.nanmax(d[:,:,:,1]))
    
    

(86407, 6, 6, 2)
4.457507133483887
1172.404296875
2.651884078979492
825.70654296875


In [96]:

slps = np.load('/home/cat/meta_data_slp.npy',allow_pickle=True)
vids = np.load('/home/cat/meta_data.npy', allow_pickle=True)

for k in range(len(slps)):
    #print ('')
    temp = slps[k]
    temp2 = vids[k]
    if temp[1]!=temp2[1]:
        print (temp)
        print (temp2)



['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_27_17_48_48_476256_compressed_Day_deleted.slp'
 '0']
['/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/input_dir/2020_09_27_17_48_48_476256_compressed_Day_deleted.slp'
 '42']


In [111]:
d = np.load('/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/Day_human_fnames.npy')
print (d.shape)

didx = np.load('/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/id_switch/cohort4/Day_human_frames_idx.npy')
print (didx.shape)


(1600,)
(1600,)
